In [ ]:
import pandas as pd

df = pd.read_csv("personnelportfoliodataset.csv")
df.head()

In [ ]:
import re

def clean_text(text):
    text = text.lower()
    return text

df["prompt"] = df["prompt"].apply(clean_text)
df["response"] = df["response"].apply(clean_text)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True,)
test_df = test_df.reset_index(drop=True,)

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

In [ ]:
def tokenize_text(example):
  input = tokenizer(example["prompt"], truncation=True, padding="max_length", max_length=500)
  target = tokenizer(example["response"], truncation=True, padding="max_length", max_length=500)

  input["labels"] = target["input_ids"]

  return input

train_df = train_df.apply(tokenize_text, axis=1)
test_df = test_df.apply(tokenize_text, axis=1)

In [ ]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [ ]:
from transformers import Trainer,TrainingArguments

trainingargs = TrainingArguments(
    output_dir="./results",          # output directory for checkpoints
    num_train_epochs=20,              # number of training epochs
    per_device_train_batch_size=8,   # batch size per device during training
    per_device_eval_batch_size=8,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir="./logs",            # directory for storing logs
    logging_steps=50,                # how often to log training info
    save_steps=500,                  # how often to save a model checkpoint
    eval_steps=50,                   # how often to run evaluation
)

trainer = Trainer(
    model=model,
    train_dataset=train_df,
    eval_dataset=test_df,
    args=trainingargs,
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: abdulraheemabdullah859 (abdulraheemabdullah859-aha-dynamics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


Step,Training Loss


In [ ]:
model.save_pretrained("./chatbot_model")
tokenizer.save_pretrained("./chatbot_model")

model = T5ForConditionalGeneration.from_pretrained("./chatbot_model")
tokenizer = T5Tokenizer.from_pretrained("./chatbot_model")

NameError: name 'tokenizer' is not defined

In [ ]:
device = model.device
def chatbot(query):
  query = clean_text(query)
  input_ids = tokenizer(query,return_tensors="pt",padding="max_length",truncation=True,max_length=250).input_ids
  outputs = model.generate(
      input_ids=input_ids,
      num_beams=5,
      early_stopping=True,
      max_length=500,
  )
  return tokenizer.decode(outputs[0],skip_special_tokens=True)

while True:
  user = input("Client : ")
  print("Abdulraheem",chatbot(user))
  if user == "exit":
    break